# Download Sentinel-3 products from Copernicus

## Introduction

Once we know the products we need from our search using STAC (see `01_search_stac_on_cdse.ipynb`), we can download the respective products from the Copernicus Data Space.

In thise notebook, we'll show how to

1. Get an acccess token
2. Get a refresh token
3. Download a number of data products

To download data, you will need to generate a user.

Note that the downloads become a bit slow after a while, probably due to throttling. There is an S3 endpoint that can be used instead, but this is not part of this notebook (yet).

Further Reading:

1. https://documentation.dataspace.copernicus.eu/Registration.html.
2. https://documentation.dataspace.copernicus.eu/APIs/Token.html
3. https://documentation.dataspace.copernicus.eu/APIs/OData.html#product-download
4. https://documentation.dataspace.copernicus.eu/APIs/S3.html

## Secret Management

You will also need a `.env` file to keep your Copernicus Dataspace username and passord, viz.

```sh
$ cat .env
CDSE_USERNAME=user@domain.tld
CDSE_PASSWORD=your_password
CDSE_CLIENT=cdse-public
```

## The Code

In [1]:
import requests
import pprint
from tqdm.auto import tqdm
from dotenv import load_dotenv
import os

/Users/volkerh/miniforge3/envs/iliad/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
CDSE_USERNAME = os.getenv("CDSE_USERNAME")
CDSE_PASSWORD = os.getenv("CDSE_PASSWORD")
CDSE_CLIENT = os.getenv("CDSE_CLIENT")
# print(CDSE_USERNAME)
# print(CDSE_PASSWORD)
# print(CDSE_CLIENT)

In [3]:
def _get_access_token_and_refresh_token_from_userpass():
    # https://documentation.dataspace.copernicus.eu/APIs/OData.html#product-download
    # https://documentation.dataspace.copernicus.eu/APIs/Token.html

    # curl --location --request POST 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token' \
    #   --header 'Content-Type: application/x-www-form-urlencoded' \
    #   --data-urlencode 'grant_type=password' \
    #   --data-urlencode 'username=<LOGIN>' \
    #   --data-urlencode 'password=<PASSWORD>' \
    #   --data-urlencode 'client_id=cdse-public'
    
    url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    data = {'grant_type': 'password',
            'username': CDSE_USERNAME,
            'password': CDSE_PASSWORD,
            'client_id': CDSE_CLIENT}
    r = requests.post(url, headers=headers, data=data)
    access_token = r.json()['access_token']
    refresh_token = r.json()['refresh_token']
    return access_token, refresh_token

access_token, refresh_token = _get_access_token_and_refresh_token_from_userpass()

In [4]:
def _get_acccess_token_from_refresh_token(refresh_token):
    # https://documentation.dataspace.copernicus.eu/APIs/OData.html#product-download
    
    # curl --location --request POST 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token' \
    #   --header 'Content-Type: application/x-www-form-urlencoded' \
    #   --data-urlencode 'grant_type=refresh_token' \
    #   --data-urlencode 'refresh_token=<REFRESH_TOKEN>' \
    #   --data-urlencode 'client_id=cdse-public'

    url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    data = {'grant_type': 'refresh_token',
            'refresh_token': refresh_token,
            'client_id': CDSE_CLIENT}
    r = requests.post(url, headers=headers, data=data)
    access_token = r.json()['access_token']
    return access_token

access_token = _get_acccess_token_from_refresh_token(refresh_token)

In [5]:
def _download_product(url, access_token):
    # https://stackoverflow.com/questions/41707229/why-is-tqdm-printing-to-a-newline-instead-of-updating-the-same-line
    # https://stackoverflow.com/questions/37573483/progress-bar-while-download-file-over-http-with-requests
    # https://documentation.dataspace.copernicus.eu/APIs/OData.html#product-download
    
    headers = {"Authorization": f"Bearer {access_token}"}
    session = requests.Session()
    session.headers.update(headers)
    response = session.get(url, headers=headers, stream=True)
    if response.status_code != 200:
        print(response.status_code)
        print(response.headers)
        print(response.json())
    total_size = int(response.headers.get("content-length", 0))
    filename = response.headers.get("content-disposition", 0).split(';')[-1].split('=')[-1]
    
    with tqdm(total=total_size, unit="B", unit_scale=True, position = 0, leave=True, ascii=False) as progress_bar:
        if os.path.exists(filename):
            #raise Exception("File %s already exists" % filename)
            print("File %s already exists... skipping." % filename)
        else:
            with open(filename, "wb") as file:
                for chunk in response.iter_content(chunk_size=8192):
                    progress_bar.update(len(chunk))
                    if chunk:
                        file.write(chunk)

In [6]:
# url = f"https://download.dataspace.copernicus.eu/odata/v1/Products(653131b1-e28e-4daa-bad1-c5c6f732b4dd)/$value"  # s2, 600 mB
# url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products(a5ab498a-7b2f-4043-ae2a-f95f457e7b3b)/$value"  # s5p, 3GB
# url = f"https://download.dataspace.copernicus.eu/odata/v1/Products(e260c4fb-6f50-532a-8a44-f62540d05100)/$value"
# url = f"https://download.dataspace.copernicus.eu/odata/v1/Products(558fc6b7-bc29-5645-844c-ef9486851e8a)/$value"
url = f"https://download.dataspace.copernicus.eu/odata/v1/Products(5e1c70ab-de2f-543e-a1e2-4a5e4c2569ec)/$value"  # s3 olci

# refresh token
access_token = _get_acccess_token_from_refresh_token(refresh_token)
_download_product(url, access_token)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 209M/209M [01:43<00:00, 2.02MB/s]
